In [1]:
import os
from pathlib import Path
from typing import Annotated

import nest_asyncio
import uvicorn
from fastapi import FastAPI, Request, WebSocket
from fastapi.responses import HTMLResponse, JSONResponse
from fastapi.staticfiles import StaticFiles
from fastapi.templating import Jinja2Templates

from autogen.agentchat.realtime_agent import RealtimeAgent, WebsocketAudioAdapter

/work/davor/projects/airt/ag2/.venv-3.9/lib/python3.9/site-packages/flaml/__init__.py:20: UserWarning: flaml.automl is not available. Please install flaml[automl] to enable AutoML functionalities.
  warnings.warn("flaml.automl is not available. Please install flaml[automl] to enable AutoML functionalities.")


In [2]:
# Configuration
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
PORT = int(os.getenv("PORT", 5050))

if not OPENAI_API_KEY:
    raise ValueError("Missing the OpenAI API key. Please set it in the .env file.")

llm_config = {
    "timeout": 600,
    "cache_seed": 45,  # change the seed for different trials
    "config_list": [
        {
            "model": "gpt-4o-realtime-preview-2024-10-01",
            "api_key": OPENAI_API_KEY,
        }
    ],
    "temperature": 0.8,
}

In [3]:
nest_asyncio.apply()

In [ ]:
app = FastAPI()

notebook_path = os.getcwd()

app.mount(
    "/static", StaticFiles(directory=Path(notebook_path) / "agentchat_realtime_websocket" / "static"), name="static"
)

# Templates for HTML responses

templates = Jinja2Templates(directory=Path(notebook_path) / "agentchat_realtime_websocket" / "templates")


@app.get("/", response_class=JSONResponse)
async def index_page():
    return {"message": "Websocket Audio Stream Server is running!"}


@app.get("/start-chat/", response_class=HTMLResponse)
async def start_chat(request: Request):
    """Endpoint to return the HTML page for audio chat."""
    port = PORT  # Extract the client's port
    return templates.TemplateResponse("chat.html", {"request": request, "port": port})


@app.websocket("/media-stream")
async def handle_media_stream(websocket: WebSocket):
    """Handle WebSocket connections providing audio stream and OpenAI."""
    await websocket.accept()

    audio_adapter = WebsocketAudioAdapter(websocket)
    realtime_agent = RealtimeAgent(
        name="Weather Bot",
        system_message="Hello there! I am an AI voice assistant powered by Autogen and the OpenAI Realtime API. You can ask me about weather, jokes, or anything you can imagine. Start by saying How can I help you?",
        llm_config=llm_config,
        audio_adapter=audio_adapter,
    )

    @realtime_agent.register_realtime_function(name="get_weather", description="Get the current weather")
    def get_weather(location: Annotated[str, "city"]) -> str:
        return "The weather is cloudy." if location == "Seattle" else "The weather is sunny."

    await realtime_agent.run()


uvicorn.run(app, host="0.0.0.0", port=PORT)

INFO:     Started server process [3287959]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:5050 (Press CTRL+C to quit)


INFO:     127.0.0.1:39902 - "GET /start-chat/ HTTP/1.1" 200 OK
INFO:     127.0.0.1:39892 - "GET /static/main.js HTTP/1.1" 304 Not Modified
INFO:     127.0.0.1:39902 - "GET /static/wavtools.js HTTP/1.1" 404 Not Found
INFO:     127.0.0.1:39902 - "GET /static/Audio.js HTTP/1.1" 304 Not Modified


INFO:     ('127.0.0.1', 39908) - "WebSocket /media-stream" [accepted]
INFO:     connection open
